In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
import en_core_web_sm

In [3]:
import nltk
nltk.download('gutenberg')
nltk.download('stopwords')

[nltk_data] Downloading package gutenberg to C:\Users\Ben
[nltk_data]     Stano\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ben
[nltk_data]     Stano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [5]:
len(alice)

141708

In [6]:
len(persuasion)

462818

In [7]:
alicetr = alice[:20000]
persuasiontr = persuasion[:20000]

In [8]:
nlp = en_core_web_sm.load()

In [9]:
alice_doc = nlp(alicetr)

In [10]:
persuasion_doc = nlp(persuasiontr)

In [11]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [12]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    df['punctuation'] = 0
    df['other punctuation'] = 0
    df.loc[:, '.'] = 0
    df.loc[:, '?'] = 0
    df.loc[:, '!'] = 0
    df.loc[:, ','] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation, stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # Get number of punctuation in a sentence
        puncs = [token.lemma_
                 for token in sentence
                 if (
                     token.is_punct
                 )]
        # Increase punctuation count by how many were use
        for punc in puncs:
            df.loc[i,'punctuation'] += 1
            try:
                df.loc[i,punc] += 1
            except:
                df.loc[i,'other punctuation'] += 1
                
            
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [13]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0


,ought,she,mindedness,drive,begin,leg,judgement,receive,march,work,...,thirteen,white,text_sentence,text_source,punctuation,other punctuation,.,?,!,","
0,0,0,0,0,1,0,0,0,0,0,...,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,10,5,0,1,0,4
1,0,0,0,0,0,0,0,0,0,0,...,0,1,"(So, she, was, considering, in, her, own, mind...",Carroll,7,3,1,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,4,2,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,"(Oh, dear, !)",Carroll,1,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,"(I, shall, be, late, !, ')",Carroll,2,1,0,0,1,0


# Creating an Accurate Model

I want to be able to predict the writer of a text with an accuracy of higher than 93%

In [14]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

## Random Forest

Now I'll give the bag of words features a whirl by trying a random forest.

In [15]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9945652173913043

Test set score: 0.8699186991869918


The RFC seems to have overfit tremendously. It will probably not be the best model.

## Logistic Regression
Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=42)
train = lr.fit(X_train, y_train)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.9891304347826086

Test set score: 0.943089430894309


Logistic regression performs a bit better than the random forest.

## Gradient Boosting

And finally, I'll see what gradient boosting can do:

In [17]:
clf = ensemble.GradientBoostingClassifier(random_state=42)
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9945652173913043

Test set score: 0.9024390243902439


## Support Vector Machine

In [18]:
from sklearn.svm import SVC

svc = SVC(C=150,random_state=42)
svc.fit(X_train, y_train)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.9891304347826086

Test set score: 0.9349593495934959


## Modeling Conclusion

Looks like logistic regression is the winner.

# Testing Modeling Accuracy on Different Texts

## Identifying Between Alice and Emma

What if I feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [19]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [20]:
emmatr = emma[:20000]

In [21]:
# Parse our cleaned data.
emma_doc = nlp(emmatr)

In [22]:
# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [23]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
done


In [24]:
y_train.reset_index(drop=True,inplace=True)

In [25]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.5868852459016394


col_0,Austen,Carroll
row_0,,
Austen,61,126
Carroll,0,118


## Importing Another Book

I want to know how my model can identify between authors it has trained on and authors it has not trained on.  To do that, I will import _Buster Bear_ by Thornton Burgess. 

In [26]:
# Clean the Emma data.
brown = gutenberg.raw('burgess-busterbrown.txt')
brown = re.sub(r'VOLUME \w+', '', brown)
brown = re.sub(r'CHAPTER \w+', '', brown)
brown = text_cleaner(brown)
print(brown[:100])

I BUSTER BEAR GOES FISHING Buster Bear yawned as he lay on his comfortable bed of leaves and watched


In [27]:
browntr = brown[:20000]

In [28]:
# Parse our cleaned data.
brown_doc = nlp(browntr)

In [29]:
# Group into sentences.
brown_sents = [[sent, "Burgess"] for sent in brown_doc.sents]

brown_sents = brown_sents[0:len(alice_sents)]

In [30]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
brown_sentences = pd.DataFrame(brown_sents)
brown_bow = bow_features(brown_sentences, common_words)

print('done')

Processing row 0
done


## Alice vs. Buster Bear

In [31]:
# Combine Alice and Brown sentences
X_browna_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    brown_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_browna_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Burgess'] * brown_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_browna_test, y_browna_test))
lr_browna_predicted = lr.predict(X_browna_test)
pd.crosstab(y_browna_test, lr_browna_predicted)


Test set score: 0.3722397476340694


col_0,Austen,Carroll
row_0,,
Burgess,36,163
Carroll,0,118


## Persuasion vs. Buster Bear

In [32]:
# Combine Persuasion and Brown Sentences
X_brownp_test = np.concatenate((
    X_train[y_train[y_train=='Austen'].index],
    brown_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_brownp_test = pd.concat([y_train[y_train=='Austen'],
                         pd.Series(['Burgess'] * brown_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_brownp_test, y_brownp_test))
lr_brownp_predicted = lr.predict(X_brownp_test)
pd.crosstab(y_brownp_test, lr_brownp_predicted)


Test set score: 0.24150943396226415


col_0,Austen,Carroll
row_0,,
Austen,64,2
Burgess,36,163


## Austen vs. Buster Bear (Burgess)

In [33]:
# Combine Persuasion and Emma sentences into all Austin sentences
X_austen_test = np.concatenate((
    X_train[y_train[y_train=='Austen'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_austen_test = pd.concat([y_train[y_train=='Austen'],
                         pd.Series(['Austen'] * emma_bow.shape[0])]) 

In [34]:
# Combine Austin and Burgess Sentences
X_brownj_test = np.concatenate((
    X_austen_test,
    brown_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_brownj_test = pd.concat([y_austen_test,
                         pd.Series(['Burgess'] * brown_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_brownj_test, y_brownj_test))
lr_brownj_predicted = lr.predict(X_brownj_test)
pd.crosstab(y_brownj_test, lr_brownj_predicted)


Test set score: 0.27654867256637167


col_0,Austen,Carroll
row_0,,
Austen,125,128
Burgess,36,163


## Cross Text Conlcusion

The model was not very good at predicting either of the authors it trained on against _Buster Bear_. The model was best at identifying between _Alice in Wonderland_ against _Buster Bear_, but not by much. 